<a href="https://colab.research.google.com/github/chanseongparkk/2022_Data_Analysis_Intern_practice/blob/main/07_04_03_%EC%9D%B4%EC%83%81%EC%B9%98%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 이상값 탐색 및 처리

### 1.라이브러리 import

In [ ]:
import random
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# os에 따른 font 깨짐 제거를 위해 font 지정

import os

if os.name == "nt":
    font_family = "Malgun Gothic"
else: #mac os
    font_family = "AppleGothic"
    
# - 값이 깨지는 문제 해결을 위해 파라미터값 설정

sns.set(font = font_family, rc = {"axes.unicode_minus" : False})

import warnings
warnings.filterwarnings("ignore")

### 2. z검정을 이용한 이상치 처리

#### 2-1 데이터 생성

In [ ]:
# 3000개의 랜덤 데이터 생성

np_data = np.random.randint(1,100,size = 3000)
data = list(np_data)

In [ ]:
data[:2]

In [ ]:
# 히스토그램으로 생성된 데이터 분포 확인

plt.hist(data)
plt.show()

In [ ]:
# data의 정규분포 확인

data_mean = np.mean(data)
data_std = np.std(data)

print("평균 : ", data_mean)
print("분산 : ", data_std)

# 확률밀도함수 생성

pdf = stats.norm.pdf(np.sort(data),data_mean,data_std)

plt.figure()
plt.plot(np.sort(data),pdf)

In [ ]:
# 표준정규분포로 변환

std_list = []

for i in data:
  z = (i-data_mean)/data_std
  std_list.append(z)


ol_data_mean = np.mean(std_list)
ol_data_std = np.std(std_list)

print("잔차 평균값: ", ol_data_mean)
print("잔차 분산값: ", ol_data_std)

ol_pdf = stats.norm.pdf(np.sort(std_list),
                        ol_data_mean, ol_data_std)

plt.figure()
plt.plot(np.sort(std_list),ol_pdf) 

#### 2-2 데이터셋에 이상값 추가

In [ ]:
ol_data = data.copy()
ol_no = 10

for i in range(ol_no):
  rand_no = random.randint(0,len(data))
  ol_data.insert(rand_no,ol_data[rand_no]*2) #이상값 생성

# 이상값이 포함된 데이터의 정규분포도
ol_data_mean = np.mean(ol_data)
ol_data_std = np.std(ol_data)

print("평균: ",ol_data_mean)
print("분산: ",ol_data_std)

ol_pdf = stats.norm.pdf(np.sort(ol_data),
                        ol_data_mean, ol_data_std)

plt.figure()
plt.plot(np.sort(ol_data), ol_pdf)

In [ ]:
# 이상값이 추가된 데이터의 표준화

ol_std_list = []

for i in ol_data:
  z = (i-ol_data_mean)/ol_data_std
  ol_std_list.append(z)

ol_st_data_mean = np.mean(ol_std_list)
ol_st_data_std = np.std(ol_std_list)

print("잔차 평균: ", ol_st_data_mean)
print("잔차 분산: ", ol_st_data_std)

ol_std_pdf = stats.norm.pdf(np.sort(ol_std_list),
                            ol_st_data_mean, ol_st_data_std)


plt.figure()
plt.plot(np.sort(ol_std_list), ol_std_pdf)



#### 2-3. Z값이 2를 넘어가는 데이터 삭제

In [ ]:
# 전체 라인 수 확인
len(ol_std_list)

In [ ]:
# -2~2 범위 넘어가면 이상값으로 제거

for i in ol_std_list:
  z = i
  if z > 2:
    ol_std_list.remove(z)
  elif z < -2:
    ol_std_list.remove(z)
  


In [ ]:
# 이상값 제거 후 라인 수 
len(ol_std_list)

In [ ]:
# 이상값 제거 후 표준전규분포 확인

ol_std_pdf = stats.norm.pdf(np.sort(ol_std_list),
                            ol_st_data_mean, ol_st_data_std)

plt.figure()
plt.plot(np.sort(ol_std_list),ol_std_pdf)

### 3. 사분위수 범위로 이상값 삭제

#### 3-1 이상값 데이터 생성

In [ ]:
# 이상값 데이터 생성

boxdata = pd.DataFrame(np.random.randn(100,3),columns = ["A열","B열","C열"])

In [ ]:
boxdata.head()

#### 3-2 이상값 데이터 확인

In [ ]:
# 기술통계정보 확인

boxdata.describe()

In [ ]:
# 박스플롯을 이용한 이상값 확인

sns.boxplot(data = boxdata)
plt.show()

#### 3-3 이상값 확인

In [ ]:
def get_outlier(df = None, columns = None, weight = 1.5):
  quantile_25 = df[columns].quantile(0.25)
  quantile_75 = df[columns].quantile(0.75)

  IQR = quantile_75-quantile_25
  IQR_weight = IQR*weight

  lowest = quantile_25-IQR_weight
  highest = quantile_75+IQR_weight

  outlier_index = df[columns][(df[columns] < lowest) | (df[columns] > highest)].index
  return outlier_index

In [ ]:
outlier_index = get_outlier(df = boxdata, columns = "B열", weight = 1.5)

In [ ]:
outlier_index

In [ ]:
# 이상값 삭제

boxdata.drop(outlier_index, axis = 0, inplace = True)

In [ ]:
boxdata.shape

In [ ]:
sns.boxplot(data = boxdata)
plt.show()

In [ ]:
# 함수 사용하여 이상값 확인
outlier_index = get_outlier(df = boxdata, columns = "A열", weight = 1.5)

outlier_index

# 이상값 삭제

boxdata.drop(outlier_index,axis = 0, inplace = True)


sns.boxplot(data = boxdata)
plt.show()

In [ ]:
# 함수 사용하여 이상값 확인
outlier_index = get_outlier(df = boxdata, columns = "C열", weight = 1.5)

outlier_index

# 이상값 삭제

boxdata.drop(outlier_index,axis = 0, inplace = True)


sns.boxplot(data = boxdata)
plt.show()


In [ ]:
boxdata.shape